In [ ]:
# %load main.py
import cPickle as pickle
import gzip
import numpy
from midi_to_statematrix import *

import multi_training
import model

def gen_adaptive(m,pcs,times,keep_thoughts=False,name="final"):
	xIpt, xOpt = map(lambda x: numpy.array(x, dtype='int8'), multi_training.getPieceSegment(pcs))
	all_outputs = [xOpt[0]]
	if keep_thoughts:
		all_thoughts = []
	m.start_slow_walk(xIpt[0])
	cons = 1
	for time in range(multi_training.batch_len*times):
		resdata = m.slow_walk_fun( cons )
		nnotes = numpy.sum(resdata[-1][:,0])
		if nnotes < 2:
			if cons > 1:
				cons = 1
			cons -= 0.02
		else:
			cons += (1 - cons)*0.3
		all_outputs.append(resdata[-1])
		if keep_thoughts:
			all_thoughts.append(resdata)
	noteStateMatrixToMidi(numpy.array(all_outputs),'output/'+name)
	if keep_thoughts:
		pickle.dump(all_thoughts, open('output/'+name+'.p','wb'))

def fetch_train_thoughts(m,pcs,batches,name="trainthoughts"):
	all_thoughts = []
	for i in range(batches):
		ipt, opt = multi_training.getPieceBatch(pcs)
		thoughts = m.update_thought_fun(ipt,opt)
		all_thoughts.append((ipt,opt,thoughts))
	pickle.dump(all_thoughts, open('output/'+name+'.p','wb'))

if __name__ == '__main__':
    # load data
    pcs = multi_training.loadPieces("music")
    # create instance model
    m = model.Model([300,300],[100,50], dropout=0.5)
    # train data
    multi_training.trainPiece(m, pcs, 10000)
    # save model weights
    pickle.dump( m.learned_config, open( "output/final_learned_config.p", "wb" ) )


Loaded waldstein_1
Loaded beethoven_opus90_2
Loaded waldstein_3
Loaded beethoven_opus10_2
Loaded beethoven_opus10_3
Loaded appass_2
Loaded appass_3
Loaded pathetique_1
Loaded mond_1
Loaded beethoven_les_adieux_2
Loaded mond_3
Loaded pathetique_2
Loaded pathetique_3
Loaded beethoven_les_adieux_1
Loaded beethoven_opus22_1
Loaded beethoven_hammerklavier_1
Loaded beethoven_opus22_4


model.py:370: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  self.walk_input: next_input
model.py:373: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  updates.update({hidden:newstate for hidden, newstate, layer in zip(self.walk_hiddens, new_states, self.time_model.layers) if has_hidden(layer)})


In [ ]:
#generate composition
gen_adaptive(m,pcs,10,name="composition")